# 日誌視覺化與攻擊序列分析系統 (Log Visualization and Attack Sequence Analysis System)

本專案用於處理 Caldera 攻擊模擬平台產生的日誌資料，將其轉換為網路圖(Graph)並進行視覺化分析。主要功能包括：

1. **資料預處理**: 將 CSV 格式的日誌檔案轉換為標準化格式
2. **圖形建構**: 基於日誌事件建立節點和邊的關係圖
3. **攻擊序列偵測**: 識別預定義的攻擊模式和行為序列
4. **互動式視覺化**: 產生可互動的 HTML 圖形介面，支援時間篩選和序列分析

## 系統架構 (System Architecture)
- **輸入**: Caldera_Ability_Statistics/ 目錄中的 CSV 日誌檔案(由 Procmon 錄製)
- **處理**: 使用 NetworkX 建立圖形結構，並應用攻擊模式比對演算法
- **輸出**: 儲存為 pickle 格式的圖形檔案和互動式 HTML 視覺化介面

# 套件匯入與初始設定 (Package Imports and Initial Configuration)

此單元格匯入所有必要的 Python 套件和模組：

## 核心套件 (Core Packages)
- **glob, os**: 檔案系統操作和路徑處理
- **networkx**: 圖形資料結構的建立與操作
- **pandas**: 資料處理和 CSV 檔案讀取
- **matplotlib**: 基礎圖形繪製功能

## 專用工具 (Specialized Tools)
- **tqdm**: 進度條顯示，用於長時間運行的迴圈
- **natsort**: 自然排序，確保檔案按正確順序處理
- **pickle**: 物件序列化，用於儲存圖形結構
- **uuid**: 產生唯一識別碼，用於節點標識

## 自定義模組 (Custom Modules)
- **graphutil**: 包含圖形生成和視覺化的核心函數

設定輸出目錄 GRAPH_PATH = "Graphs" 用於儲存處理後的圖形檔案。

# Define packages

In [1]:
from glob import glob
from tqdm import tqdm
import networkx as nx
from natsort import natsorted
import json
import uuid
import os
import pickle
import matplotlib.pyplot as plt
from datetime import datetime
import pandas as pd
import re
from collections import defaultdict, namedtuple

from graphutil import *

GRAPH_PATH = "Graphs"

# 攻擊序列模式定義 (Attack Sequence Pattern Definition)

本單元格定義了系統用於識別攻擊行為的預設模式。每個模式使用 `SequencePattern` 命名元組(namedtuple)結構，包含以下屬性：

## 模式屬性說明 (Pattern Attributes)
- **name**: 模式名稱，用於識別攻擊類型
- **operations**: 操作序列列表，定義該攻擊模式包含的系統操作
- **color**: 視覺化時使用的顏色代碼
- **description**: 模式的文字描述
- **min_length**: 最小序列長度要求
- **strict_order**: 是否要求嚴格的操作順序
- **results**: 預期的操作結果狀態

## 預定義攻擊模式 (Predefined Attack Patterns)

### 1. 程序創建模式 (Process Creation)
- 偵測新程序的創建行為
- 顏色：紅色 (#FF3333)

### 2. 檔案操作模式 (File Manipulation)
- **File_Creation_Write**: 檔案創建後進行寫入
- **File_Creation_Metadata_Write**: 包含元資料查詢的完整檔案操作
- 顏色：橙色/黃色系

### 3. 註冊表操作模式 (Registry Manipulation)
- **Registry_Creation_Modification**: 註冊表鍵值的創建和修改
- **Registry_Modification**: 現有註冊表鍵值的修改
- 顏色：青色/綠色系

### 4. 網路通訊模式 (Network Communication)
- **TCP_Communication**: TCP 連接的完整生命週期
- 包含連接、發送、接收、斷開等步驟
- 顏色：藍色 (#0066FF)

這些模式將在後續的圖形分析中用於自動識別和標記可疑的攻擊行為序列。

# Rules for the sequence matching

In [2]:
# Predefined attack sequence patterns based on exact operation names
SequencePattern = namedtuple('SequencePattern', ['name', 'operations', 'color', 'description', 'min_length', 'strict_order', 'results'])


ATTACK_SEQUENCE_PATTERNS = [
    # Process creation patterns
    SequencePattern(
        name="Process_Creation",
        operations=["Process Create"],
        color="#FF3333",  # Red
        description="Process creation operations",
        min_length=1,
        strict_order=False,
        results=["SUCCESS"]  # Expected results for this pattern
    ),

    # File manipulation patterns
    SequencePattern(
        name="File_Creation_Write",
        operations=["CreateFile", "WriteFile"],
        color="#FF6600",  # Orange
        description="File creation followed by write operations",
        min_length=2,
        strict_order=True,
        results=["SUCCESS", "SUCCESS"]  
    ),
    SequencePattern(
        name="File_Creation_Metadata_Write",
        operations=["CreateFile", "QueryBasicInformationFile", "WriteFile", "CloseFile"],
        color="#FAD000",  # Yellow
        description="File creation with metadata query and write",
        min_length=1,
        strict_order=False,
        results=["SUCCESS", "SUCCESS", "SUCCESS", "SUCCESS"]  
    ),

    # Registry manipulation patterns
    SequencePattern(
        name="Registry_Creation_Modification",
        operations=["RegCreateKey", "RegSetValue", "RegQueryKey", "RegCloseKey"],
        color="#34CCFF",  # Cyan
        description="Registry key Create and modification",
        min_length=2,
        strict_order=False,
        results=["SUCCESS", "SUCCESS", "SUCCESS", "SUCCESS"]  
    ),

    SequencePattern(
        name="Registry_Modification",
        operations=["RegOpenKey", "RegSetValue", "RegQueryKey", "RegCloseKey"],
        color="#33CC33",  # Green
        description="Registry key open and modification",
        min_length=2,
        strict_order=False,
        results=["SUCCESS", "SUCCESS", "SUCCESS", "SUCCESS"]  
    ),

    # Network communication patterns
    SequencePattern(
        name="TCP_Communication",
        operations=["TCP Connect", "TCP Send", "TCP Receive", "TCP Disconnect"],
        color="#0066FF",  # Blue
        description="TCP network communication sequence",
        min_length=2,
        strict_order=True,
        results=["SUCCESS", "SUCCESS", "SUCCESS", "SUCCESS"] 
    ),
    
]

# 日誌檔案轉換為圖形結構 (Convert Log Files to Graph Structure)

此區塊負責將 CSV 格式的日誌檔案轉換為網路圖(NetworkX Graph)結構，是整個系統的核心資料處理步驟。

## 主要處理流程 (Main Processing Flow)

### 1. 檔案探索與驗證 (File Discovery and Validation)
- 搜尋 `Caldera_Ability_Statistics/` 目錄中的 CSV 檔案
- 優先處理已有 `_raw_events_with_lineid.csv` 後綴的檔案
- 若不存在，則自動為原始 CSV 檔案添加 `lineid` 欄位

### 2. 編碼處理 (Encoding Handling)
系統嘗試多種字元編碼以確保相容性：
- UTF-8 (標準編碼)
- UTF-8-BOM (包含位元組順序標記)
- CP950 (繁體中文 Big5 編碼)

### 3. 資料結構轉換 (Data Structure Conversion)
將 CSV 資料列轉換為 `campaign_events` 格式，包含：
- **srcNode**: 來源節點資訊 (通常為程序)
- **dstNode**: 目標節點資訊 (檔案、註冊表、網路等)
- **relation**: 操作關係類型
- **timestamp**: 時間戳記
- **line_id**: 原始日誌行號

### 4. 節點類型分類 (Node Type Classification)
根據事件類別自動分類節點：
- **Process**: 程序節點，包含 PID 和程序名稱
- **File**: 檔案系統節點，包含路徑資訊
- **Registry**: 註冊表節點，包含鍵值路徑
- **Network**: 網路節點，包含 IP 位址資訊

### 5. UUID 生成策略 (UUID Generation Strategy)
使用 UUID5 (基於命名空間的雜湊) 確保相同資源的唯一性：
- 程序: `程序名稱_PID`
- 檔案: `檔案路徑`
- 註冊表: `註冊表鍵值路徑`
- 網路: `IP位址`

這種方法確保了圖形中相同資源的節點會被正確合併，避免重複節點的產生。

# Convert all logs into graphs

In [3]:
# caldera_dir = os.path.join(os.getcwd(), 'Caldera_Ability_Statistics')
# for file in os.listdir(caldera_dir):
#     if file.endswith("_raw_events_with_lineid.csv"):
#         # remove the old processed files
#         os.remove(os.path.join(caldera_dir, file))

# 檔案準備與預處理 (File Preparation and Preprocessing)

此區塊負責準備和預處理輸入的 CSV 日誌檔案，確保資料格式的統一性。

## 處理邏輯 (Processing Logic)

### 1. 檔案搜尋策略 (File Search Strategy)
- **優先級 1**: 尋找已存在的 `*_raw_events_with_lineid.csv` 檔案
- **優先級 2**: 若無預處理檔案，則處理原始 CSV 檔案並添加行號

### 2. 自動格式轉換 (Automatic Format Conversion)
當找不到預處理檔案時，系統會：
- 掃描 `Caldera_Ability_Statistics/` 目錄
- 為每個原始 CSV 檔案添加 `lineid` 欄位
- 處理多種字元編碼問題 (UTF-8, UTF-8-BOM, CP1252, ISO-8859-1, CP950)
- 將轉換後的檔案儲存為 `*_raw_events_with_lineid.csv` 格式

### 3. 錯誤處理機制 (Error Handling)
- 自動嘗試多種編碼格式
- 跳過無法解碼的檔案並記錄錯誤
- 提供詳細的處理進度回饋

### 4. 資料完整性檢查 (Data Integrity Check)
- 驗證必要欄位的存在性
- 自動添加缺失的 `lineid` 欄位
- 確保資料格式符合後續處理需求

這個預處理步驟確保了所有輸入檔案都具有統一的格式，包含必要的行號資訊，為後續的圖形建構提供可靠的資料基礎。

In [4]:
# look for CSV files that match the attached dataset pattern
csv_files = natsorted(glob(os.path.join(os.getcwd(), 'Caldera_Ability_Statistics/*_raw_events_with_lineid.csv')))
if csv_files:
    files = csv_files
    print(f"Found existing *_raw_events_with_lineid.csv files: {len(files)}")
else:
    # create *_raw_events_with_lineid.csv list from directory
    print("No *_raw_events_with_lineid.csv files found. Converting existing CSV files...")
    files = []
    
    caldera_dir = os.path.join(os.getcwd(), 'Caldera_Ability_Statistics')
    if not os.path.exists(caldera_dir):
        os.makedirs(caldera_dir)
        print(f"Created directory: {caldera_dir}")
        print("Please place your CSV files in this directory and run again.")
    else:
        for file in os.listdir(caldera_dir):
            if file.endswith(".csv") and not file.endswith("_raw_events_with_lineid.csv"):
                original_file = os.path.join(caldera_dir, file)
                
                # Read the original CSV with encoding handling
                try:
                    # Try different encodings
                    encodings_to_try = ['utf-8', 'utf-8-sig', 'cp1252', 'iso-8859-1', 'cp950']
                    df = None
                    
                    for encoding in encodings_to_try:
                        try:
                            df = pd.read_csv(original_file, encoding=encoding)
                            break
                        except UnicodeDecodeError:
                            continue
                    
                    if df is None:
                        print(f"  ❌ Could not decode {file} with any supported encoding")
                        continue
                    
                    # Add lineid column if it doesn't exist
                    if 'lineid' not in df.columns and 'LineID' not in df.columns and 'line_id' not in df.columns:
                        df['lineid'] = range(1, len(df) + 1)
                        # print(f"  Added lineid column to {file}")
                    
                    # Generate new filename
                    base_name = file.replace('.csv', '')
                    new_filename = f"{base_name}_raw_events_with_lineid.csv"
                    new_filepath = os.path.join(caldera_dir, new_filename)
                    
                    # Save the modified CSV with UTF-8 encoding
                    df.to_csv(new_filepath, index=False, encoding='utf-8')
                    files.append(new_filepath)
                    print(f"\r✅ Created: {new_filename}", end='', flush=True)
                    
                except Exception as e:
                    print(f"\r❌ Error processing {file}: {e}", end='', flush=True)
                    continue
        
        if not files:
            print("\nNo CSV files found to convert. Please add CSV files to Caldera_Ability_Statistics/ directory.")
        else:
            print(f"\nSuccessfully converted {len(files)} files to *_raw_events_with_lineid.csv format")

No *_raw_events_with_lineid.csv files found. Converting existing CSV files...
✅ Created: fe8fdc0d-5ca1-48d4-bfa1-2d3fe719686b_raw_events_with_lineid.csv
Successfully converted 167 files to *_raw_events_with_lineid.csv format


# 批次圖形生成與儲存 (Batch Graph Generation and Storage)

此核心處理單元格負責將預處理的 CSV 檔案轉換為 NetworkX 圖形結構，並儲存相關的中繼資料。

## 主要處理步驟 (Main Processing Steps)

### 1. 檔案迴圈處理 (File Loop Processing)
- 使用 `tqdm` 顯示整體處理進度
- 逐一處理每個 `*_raw_events_with_lineid.csv` 檔案
- 自動跳過無法解碼的檔案

### 2. CSV 資料解析 (CSV Data Parsing)
對每個 CSV 檔案執行以下操作：
- **多編碼嘗試**: UTF-8, UTF-8-BOM, CP950
- **欄位對應**: 將 CSV 欄位對應到標準化的事件結構
- **UUID 生成**: 為每個資源生成唯一標識符

### 3. 事件分類與節點建立 (Event Classification and Node Creation)

#### 檔案系統事件 (File System Events)
- 識別關鍵字: 'file', 'file system'
- 節點屬性: UUID, Name, Type="File"
- 路徑處理: 從 'Path' 或 'Content' 欄位提取

#### 註冊表事件 (Registry Events)
- 識別關鍵字: 'registry'
- 節點屬性: UUID, Key, Type="Registry"
- 鍵值處理: 從 'Path' 或 'Detail' 欄位提取

#### 網路事件 (Network Events)
- 識別關鍵字: 'network'
- 節點屬性: UUID, Dstaddress, Type="Network"
- IP 提取: 使用正規表達式提取 IP 位址

#### 程序事件 (Process Events)
- 識別關鍵字: 'process'
- 特殊處理: 區分 Process Create 和 Process Start
- PID 解析: 從 Detail 欄位或 PID 欄位提取程序 ID

### 4. 時間戳記處理 (Timestamp Processing)
嘗試多種時間格式：
- `"%m/%d/%Y %I:%M:%S %p"` (美式日期時間)
- `"%Y-%m-%d %H:%M:%S"` (ISO 格式)
- 失敗時使用行號作為序列號

### 5. 圖形生成與儲存 (Graph Generation and Storage)
- 調用 `generate_query_graph()` 函數建立 NetworkX 圖形
- 將圖形儲存為 pickle 格式 (`*.pkl`)
- 將邊的中繼資料儲存為 JSON 格式 (`*_edge_metadata.json`)

### 6. 中繼資料結構 (Metadata Structure)
邊的中繼資料包含：
- `line_id`: 原始日誌行號
- `operation`: 操作類型
- `timestamp`: 時間戳記
- `technique`: 技術分類
- `src_process`: 來源程序資訊
- `src_pid`: 來源程序 ID
- `dst_resource`: 目標資源
- `dst_type`: 目標類型

此步驟的輸出為每個日誌檔案產生一組完整的圖形資料，包括圖形結構和詳細的中繼資料，為後續的分析和視覺化提供基礎。

In [ ]:
print(f"Found {len(files)} file(s) to process")
print("Building provenance graphs...")

for idx, f in enumerate(tqdm(files, desc="Processing files")):
    # print(f"\n📁 Processing file {idx+1}/{len(files)}: {os.path.basename(f)}")
    
    if True:
        # Convert CSV rows to the campaign_events format expected by generate_query_graph
        import csv
        from datetime import datetime
        import re
        campaign_events = []
        
        # Try different encodings to handle various CSV formats
        encodings_to_try = ['utf-8', 'utf-8-sig', 'cp950']
        
        for encoding in encodings_to_try:
            try:
                with open(f, newline='', encoding=encoding) as csvfile:
                    reader = csv.DictReader(csvfile)
                    rows = list(reader)
                    break  # If successful, break out of the encoding loop
            except UnicodeDecodeError:
                continue  # Try next encoding
        else:
            # If all encodings fail, skip this file
            print(f"❌ Could not decode file {os.path.basename(f)} with any supported encoding")
            continue
            
        for i, row in enumerate(tqdm(rows, desc="Converting CSV", leave=False)):
            src_name = row.get('Process Name') or row.get('Image Path') or row.get('User') or 'unknown'
            pid = row.get('PID') or row.get('Parent PID') or ''
            try:
                pid_int = int(pid)
            except:
                pid_int = 0
            src_uuid = str(uuid.uuid5(uuid.NAMESPACE_DNS, f"{src_name}_{pid_int}"))
            srcNode = {"UUID": src_uuid, "Name": src_name, "Pid": pid_int, "Type": "Process"}

            event_class = (row.get('Event Class') or '').lower()
            operation = row.get('Operation') or row.get('srcEvent') or 'unknown'
            relation = operation
            dstNode = None
            if 'file' in event_class or 'file system' in event_class:
                path = row.get('Path') or row.get('Content') or ''
                dst_uuid = str(uuid.uuid5(uuid.NAMESPACE_DNS, path)) if path else src_uuid
                dstNode = {"UUID": dst_uuid, "Name": os.path.basename(path) if path else path, "Type": "File"}
            elif 'registry' in event_class:
                key = row.get('Path') or row.get('Detail') or ''
                dst_uuid = str(uuid.uuid5(uuid.NAMESPACE_DNS, key)) if key else src_uuid
                dstNode = {"UUID": dst_uuid, "Key": key, "Type": "Registry"}
            elif 'network' in event_class:
                addr = ''
                for field in ('Detail','Path','Content','srcEvent'):
                    text = row.get(field,'')
                    if not text:
                        continue
                    m = re.search(r'(?:\b)(?:\d{1,3}\.){3}\d{1,3}(?:\b)', text)
                    if m:
                        addr = m.group(0); break
                if addr:
                    dst_uuid = str(uuid.uuid5(uuid.NAMESPACE_DNS, addr))
                    dstNode = {"UUID": dst_uuid, "Dstaddress": addr, "Type": "Network"}
            elif 'process' in event_class:
                path = row.get('Path') or row.get('Image Path') or ''
                dst_name = os.path.basename(path.replace('\\', '/').strip()) if path else ''
                dst_name = dst_name.lstrip('/\\').strip()
                #print(f"Processing process event: {dst_name} from {path}")
                op_lower = (operation or '').lower()

                # Determine destination PID based on operation subtype
                dst_pid = 0
                if 'create' in op_lower:  # Process Create
                    details_text = row.get('Detail') or row.get('Details') or ''
                    m = re.search(r'PID\s*:\s*(\d+)', details_text)
                    if m:
                        try:
                            dst_pid = int(m.group(1))
                        except:
                            dst_pid = 0
                    else:
                        # Fallback to PID / Parent PID
                        dst_pid_val = row.get('PID') or row.get('Parent PID') or 0
                        try:
                            dst_pid = int(dst_pid_val)
                        except:
                            dst_pid = 0
                else:  # Process Start or others
                    dst_pid_val = row.get('PID') or row.get('Parent PID') or 0
                    try:
                        dst_pid = int(dst_pid_val)
                    except:
                        dst_pid = 0
                #print(f"  Destination PID: {dst_pid}")
                dst_uuid = str(uuid.uuid5(uuid.NAMESPACE_DNS, f"{dst_name}_{dst_pid}"))
                dstNode = {"UUID": dst_uuid, "Name": dst_name, "Pid": dst_pid, "Type": "Process"}

            # timestamp parsing: try common formats, otherwise use row index
            ts = None
            dtstr = row.get('Date & Time') or row.get('Date') or ''
            try:
                ts = int(datetime.strptime(dtstr, "%m/%d/%Y %I:%M:%S %p").timestamp())
            except:
                try:
                    ts = int(datetime.strptime(dtstr, "%Y-%m-%d %H:%M:%S").timestamp())
                except:
                    ts = i

            # include line id if present in CSV
            line_id = row.get('lineid') or row.get('LineID') or row.get('line_id') or ''

            ev = {"srcNode": srcNode, "dstNode": dstNode, "relation": relation, "timestamp": ts, "line_id": str(line_id) if line_id != '' else None}
            ev["label"] = f"CSV_{operation}"
            campaign_events.append(ev)

    # print(f"  ✅ Loaded {len(campaign_events)} events")

    eventname = os.path.basename(f).replace('_raw_events_with_lineid.csv', '')
    save_path = os.path.join(GRAPH_PATH, eventname)
    if not os.path.isdir(save_path):
        os.makedirs(save_path)

    G, edge_metadata = generate_query_graph(campaign_events)
    

    # Save graph and metadata
    with open(f"{save_path}/{eventname}.pkl", "wb") as fp:
        pickle.dump(G, fp)
    with open(f"{save_path}/{eventname}_edge_metadata.json", "w") as fp:
        # Convert edge metadata to JSON-serializable format
        json_metadata = {}
        for (src, dst, key), data in edge_metadata.items():
            edge_id = f"{src}→{dst}#{key}"
            json_metadata[edge_id] = {
                'line_id': data['line_id'],
                'operation': data['operation'],
                'timestamp': data['timestamp'],
                'technique': data['technique'],
                'src_process': data['src_process'],
                'src_pid': data['src_pid'],
                'dst_resource': data['dst_resource'],
                'dst_type': data['dst_type']
            }
        json.dump(json_metadata, fp, indent=2)

Found 167 file(s) to process
Building provenance graphs...


Processing files:  29%|██▉       | 49/167 [01:19<02:08,  1.09s/it] 

# 圖形檔案蒐集與驗證 (Graph File Collection and Validation)

此單元格負責蒐集所有已生成的圖形檔案，並驗證其完整性，為後續的視覺化處理做準備。

## 功能說明 (Functionality Description)

### 1. 目錄結構掃描 (Directory Structure Scanning)
- 掃描 `GRAPH_PATH` (預設為 "Graphs") 目錄下的所有子目錄
- 每個子目錄對應一個處理過的日誌檔案
- 目錄名稱通常為原始檔案的基礎名稱 (不含副檔名)

### 2. Pickle 檔案驗證 (Pickle File Validation)
- 在每個子目錄中尋找 `{eventname}.pkl` 檔案
- 這些 pickle 檔案包含完整的 NetworkX 圖形物件
- 只有存在對應 pickle 檔案的目錄才會被加入處理清單

### 3. 可用圖形登錄 (Available Graphs Registration)
- 將找到的有效圖形檔案路徑加入 `available_graphs` 清單
- 提供即時的發現回饋，顯示找到的圖形檔案
- 為後續的批次視覺化處理建立檔案清單

### 4. 品質保證 (Quality Assurance)
此步驟確保：
- 所有圖形檔案都是完整且可讀取的
- 避免處理不完整或損壞的資料
- 提供清楚的處理範圍資訊

這個驗證步驟是批次視覺化處理的重要前置作業，確保後續處理的可靠性和效率。

# Gather all graphs for visualization

In [ ]:
graph_base_dir = GRAPH_PATH  
available_graphs = []

# Look for .pkl files in subdirectories of Graphs/
for subdir in os.listdir(graph_base_dir):
    subdir_path = os.path.join(graph_base_dir, subdir)
    if os.path.isdir(subdir_path):
        # Look for {eventname}.pkl in each subdirectory
        pkl_file = os.path.join(subdir_path, f"{subdir}.pkl")
        if os.path.exists(pkl_file):
            available_graphs.append(pkl_file)
            print(f"\rFound graph: {pkl_file}", end='', flush=True)

FileNotFoundError: [WinError 3] 系統找不到指定的路徑。: 'Graphs'

# 互動式視覺化生成 (Interactive Visualization Generation)

此單元格為所有可用的圖形生成互動式的 HTML 視覺化介面。目前專注於基於條目範圍的互動式視覺化。

## 處理流程 (Processing Workflow)

### 1. 批次處理設置 (Batch Processing Setup)
- 處理所有在上一步驟中找到的有效圖形檔案
- 初始化成功和失敗的視覺化計數器
- 提供詳細的處理進度報告

### 2. 必要檔案檢查 (Required Files Validation)
對每個圖形檔案，系統驗證以下檔案的存在：
- **圖形檔案**: `{eventname}.pkl` (NetworkX 圖形物件)
- **中繼資料檔案**: `{eventname}_edge_metadata.json` (邊的詳細資訊)
- **預測檔案**: `Caldera_Ability_Predictions/{eventname}.csv` (可選的惡意行為預測)

### 3. 時間範圍分析 (Time Range Analysis)
- 使用 `get_graph_time_range()` 函數分析圖形的時間跨度
- 計算最小和最大時間戳記
- 跳過沒有時間資訊的圖形

### 4. 惡意行為預測整合 (Malicious Behavior Prediction Integration)
支援多種預測檔案格式：

#### 格式 A: LineID + Type + Score
- 欄位: `LineID`, `Type`, `Score`
- 條件: `Score >= 1.0`
- 類型: `src`, `dst`, 或 `both`

#### 格式 B: line_id + prediction/label
- 欄位: `line_id`, `prediction`/`is_malicious`/`label`
- 條件: 預測值為 1 或標籤為 'malicious'

### 5. 視覺化生成 (Visualization Generation)
- 調用 `create_interactive_entry_visualization()` 生成互動式 HTML
- 支援基於條目序號的篩選功能
- 整合惡意行為標記 (如果有預測資料)

### 6. 統計資訊收集 (Statistics Collection)
為每個成功的視覺化收集：
- **節點數量**: 圖形中的節點總數
- **邊數量**: 圖形中的邊總數
- **時間跨度**: 事件的時間範圍
- **惡意標記數量**: REAPr 或其他預測系統標記的惡意行為數量
- **預測可用性**: 是否有可用的預測資料

### 7. 結果報告 (Results Reporting)
生成詳細的處理報告，包括：
- 成功處理的視覺化數量和詳細資訊
- 失敗的視覺化及其原因
- 每個圖形的統計摘要表格

## 注意事項 (Notes)
- **時間視覺化**: 目前已停用時間範圍篩選功能，因為在大多數情況下用處有限
- **條目視覺化**: 專注於基於日誌條目順序的互動式分析
- **錯誤處理**: 自動跳過有問題的檔案並記錄錯誤原因

此功能為分析師提供了強大的互動式工具，可以深入探索攻擊序列和系統行為模式。

# Generate visualization that can be filtered by time and entry range

Time range is disabled for now since its almost useless

In [6]:

# Create Interactive Time Visualizations for ALL Available Graphs

print(f"🔍 Creating interactive time visualizations for {len(available_graphs)} graphs")
print("=" * 80)

successful_visualizations = []
failed_visualizations = []

for graph_file in available_graphs:
    graph_basename = os.path.basename(os.path.dirname(graph_file))
    graph_dir = os.path.dirname(graph_file)
    edge_metadata_file = os.path.join(graph_dir, f"{graph_basename}_edge_metadata.json")
    predictions_file = f"Caldera_Ability_Predictions/{graph_basename}.csv"
    
    print(f"\n📊 Processing: {graph_basename}")
    
    # Check if required files exist
    if not os.path.exists(edge_metadata_file):
        print(f"❌ Missing edge metadata file - skipping")
        failed_visualizations.append((graph_basename, "Missing metadata"))
        continue
    
    try:
        # Load the graph and metadata
        G = pickle.load(open(graph_file, "rb"))
        with open(edge_metadata_file, "r") as fp:
            edge_metadata_json = json.load(fp)
        
        # Convert edge metadata
        edge_metadata = {}
        for edge_id, data in edge_metadata_json.items():
            parts = edge_id.split('→')
            if len(parts) == 2:
                src = parts[0]
                dst_key = parts[1].split('#')
                if len(dst_key) == 2:
                    dst = dst_key[0]
                    key = int(dst_key[1]) if dst_key[1].isdigit() else dst_key[1]
                    edge_metadata[(src, dst, key)] = data
        
        # Check if graph has timestamps
        min_ts, max_ts = get_graph_time_range(G, edge_metadata)
        if min_ts is None:
            print(f"   ⚠️  No timestamps found - skipping time visualization")
            failed_visualizations.append((graph_basename, "No timestamps"))
            continue

        total_edges = G.number_of_edges()
        if total_edges == 0:
            print(f"   ⚠️  No edges found - skipping entry visualization")
            failed_entry_visualizations.append((graph_basename, "No edges"))
            continue
        
        # Load predictions if available
        MALICIOUS_SPECS = []
        has_predictions = False
        
        if os.path.exists(predictions_file):
            predictions_df = pd.read_csv(predictions_file)
            has_predictions = True
            
            # Handle different CSV formats
            if 'LineID' in predictions_df.columns and 'Type' in predictions_df.columns and 'Score' in predictions_df.columns:
                malicious_rows = predictions_df[predictions_df['Score'] >= 1.0]
                for _, row in malicious_rows.iterrows():
                    line_id = str(row['LineID'])
                    prediction_type = row['Type'].lower()
                    if prediction_type in ['src', 'dst']:
                        MALICIOUS_SPECS.append((line_id, prediction_type))
                    else:
                        MALICIOUS_SPECS.append((line_id, "both"))
            elif 'line_id' in predictions_df.columns:
                malicious_rows = predictions_df[
                    (predictions_df.get('prediction', 0) == 1) |
                    (predictions_df.get('is_malicious', False) == True) |
                    (predictions_df.get('label', 'benign').str.lower() == 'malicious')
                ]
                for _, row in malicious_rows.iterrows():
                    line_id = str(row['line_id'])
                    MALICIOUS_SPECS.append((line_id, "both"))
        
        print(f"   Entry Count: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")
        print(f"   Time range: {datetime.fromtimestamp(min_ts).strftime('%H:%M:%S')} to {datetime.fromtimestamp(max_ts).strftime('%H:%M:%S')}")
        print(f"   Provided Malicious specs: {len(MALICIOUS_SPECS)}")
        
        # Create interactive visualization
        output_time_path = os.path.join(graph_dir, f"interactive_time_{graph_basename}.html")
        output_entry_path = os.path.join(graph_dir, f"interactive_entry_{graph_basename}.html")
        
        # interactive_path = create_interactive_time_visualization(
        #     G, edge_metadata, MALICIOUS_SPECS if MALICIOUS_SPECS else None, 
        #     output_path=output_path
        # )
        interactive_path = create_interactive_entry_visualization(
            G, edge_metadata, MALICIOUS_SPECS if MALICIOUS_SPECS else None, 
            output_path=output_entry_path
        )


        # Calculate some time statistics
        total_duration = max_ts - min_ts
        seconds = total_duration
        
        successful_visualizations.append({
            'graph': graph_basename,
            'path': interactive_path,
            'nodes': G.number_of_nodes(),
            'edges': G.number_of_edges(),
            'duration_seconds': seconds,
            'malicious_specs': len(MALICIOUS_SPECS),
            'has_predictions': has_predictions
        })
        
        print(f"   ✅ Interactive visualization created: {os.path.basename(interactive_path)}")
        print(f"   📊 Duration: {seconds:.1f} seconds, REAPr: {'Yes' if MALICIOUS_SPECS else 'No'}")
        
    except Exception as e:
        print(f"   ❌ Error: {e}")
        failed_visualizations.append((graph_basename, str(e)))
        continue

print(f"\nSuccessful: {len(successful_visualizations)} | Failed: {len(failed_visualizations)}")

if successful_visualizations:
    print(f"\nSUCCESSFUL VISUALIZATIONS:")
    print(f"{'Graph':<35} {'Nodes':<8} {'Edges':<8} {'Duration':<12} {'REAPr':<8} {'File'}")
    print("-" * 90)
    
    for viz in successful_visualizations:
        duration_str = f"{viz['duration_seconds']:.1f}s"
        reapr_str = "Yes" if viz['malicious_specs'] > 0 else "No"
        filename = os.path.basename(viz['path'])
        print(f"{viz['graph']:<35} {viz['nodes']:<8} {viz['edges']:<8} {duration_str:<12} {reapr_str:<8} {filename}")
    
    
if failed_visualizations:
    print(f"\n❌ FAILED VISUALIZATIONS:")
    for graph, reason in failed_visualizations:
        print(f"   {graph}: {reason}")

🔍 Creating interactive time visualizations for 167 graphs

📊 Processing: 00e4883d-a791-4e87-8786-0c9fc7ba2478
   Entry Count: 160 nodes, 602 edges
   Time range: 21:24:07 to 21:24:07
   Provided Malicious specs: 0
Total entries (edges): 602
   Entry Count: 160 nodes, 602 edges
   Time range: 21:24:07 to 21:24:07
   Provided Malicious specs: 0
Total entries (edges): 602
Graphs\00e4883d-a791-4e87-8786-0c9fc7ba2478\interactive_entry_00e4883d-a791-4e87-8786-0c9fc7ba2478.html
Graphs\00e4883d-a791-4e87-8786-0c9fc7ba2478\interactive_entry_00e4883d-a791-4e87-8786-0c9fc7ba2478.html
Interactive entry visualization saved to: Graphs\00e4883d-a791-4e87-8786-0c9fc7ba2478\interactive_entry_00e4883d-a791-4e87-8786-0c9fc7ba2478.html
Features:
  - Entry range selector (start/end entry numbers)
  - Quick preset windows (10, 25, 50, 100, 250, 500 entries)
  - Last N entries, middle entries options
  - Sliding window navigation (Next/Previous)
  - Real-time filtering of edges and nodes
  - Shows only conne

KeyboardInterrupt: 

# 攻擊序列模式視覺化 (Attack Sequence Pattern Visualization)

此單元格專門針對特定檔案進行深度的攻擊序列模式分析和視覺化，展示系統如何識別和標記預定義的攻擊行為模式。

## 核心功能 (Core Functions)

### 1. 目標檔案設定 (Target File Configuration)
- **目標檔案**: `cfb61005899996469ae3023796792ca5` (可調整)
- 載入對應的圖形檔案 (`.pkl`) 和中繼資料 (`.json`)
- 整合預測檔案中的惡意行為標記

### 2. 序列群組偵測 (Sequence Group Detection)
調用 `find_sequence_groups()` 函數執行智慧型模式比對：

#### 參數設定 (Parameter Configuration)
- **圖形物件 (G)**: NetworkX 圖形結構
- **邊中繼資料 (edge_metadata)**: 包含操作詳細資訊
- **攻擊模式 (ATTACK_SEQUENCE_PATTERNS)**: 預定義的攻擊序列
- **最大間隔 (max_gap)**: 序列中允許的最大操作間隔
- **目標檔案 (target_file)**: 用於偵錯和日誌記錄

#### 偵測演算法 (Detection Algorithm)
- 尋找符合預定義模式的操作序列
- 計算模式匹配的置信度分數
- 評估序列的完整性和順序性
- 為每個找到的群組分配唯一的顏色標記

### 3. 偵測結果分析 (Detection Results Analysis)
系統提供詳細的偵測報告：

#### 群組資訊 (Group Information)
- **Group ID**: 群組唯一識別碼
- **Pattern Name**: 匹配的攻擊模式名稱
- **Description**: 模式的詳細描述
- **Confidence Score**: 匹配置信度 (0.0-1.0)
- **Edge Count**: 群組包含的邊數量
- **Matched Operations**: 實際匹配的操作序列
- **Expected Pattern**: 預期的操作模式
- **Completeness**: 模式完整性評分

### 4. 序列群組視覺化 (Sequence Group Visualization)
調用 `create_sequence_grouped_visualization()` 生成專門的視覺化：

#### 視覺化特色 (Visualization Features)
- **顏色編碼**: 每種攻擊模式使用特定顏色
- **群組高亮**: 屬於同一攻擊序列的邊使用相同顏色
- **未分組邊**: 不屬於任何模式的邊以灰色顯示
- **互動式介面**: 支援滑鼠懸停查看詳細資訊

### 5. 覆蓋率分析 (Coverage Analysis)
系統計算並報告模式覆蓋統計：

#### 統計指標 (Statistical Metrics)
- **群組化邊數**: 被歸類到攻擊模式的邊數量
- **總邊數**: 圖形中的邊總數
- **覆蓋率百分比**: 群組化邊占總邊數的比例
- **未群組化邊數**: 未匹配任何模式的邊數量

#### 模式分佈分析 (Pattern Distribution Analysis)
- 每種攻擊模式匹配的邊數量
- 各模式在整體圖形中的占比
- 按邊數量排序的模式重要性排名

### 6. 錯誤處理與偵錯 (Error Handling and Debugging)
- 完整的例外處理和錯誤報告
- 詳細的堆疊追蹤資訊
- 檔案存在性驗證

## 應用價值 (Application Value)
此功能對網路安全分析特別重要：
- **攻擊路徑視覺化**: 清楚展示攻擊者的行為序列
- **模式識別**: 自動識別已知的攻擊技術
- **異常偵測**: 突出顯示可疑的行為模式
- **事件關聯**: 將相關的系統事件群組化

這種基於模式的分析方法有助於安全分析師快速理解複雜的攻擊場景，並識別潛在的威脅行為。

# Generate visualization that colors the found patterns

# 單一檔案深度序列分析 (Single File Deep Sequence Analysis)

此最終單元格展示如何對特定的日誌檔案執行深度的攻擊序列分析，結合了圖形載入、模式偵測、預測整合和專門的序列視覺化。

## 分析流程 (Analysis Workflow)

### 1. 檔案初始化 (File Initialization)
- **目標檔案**: `cfb61005899996469ae3023796792ca5` (範例檔案)
- 構建完整的檔案路徑結構
- 驗證所有必要檔案的存在性

### 2. 資料載入與整合 (Data Loading and Integration)

#### 圖形資料載入 (Graph Data Loading)
- 載入 pickle 格式的 NetworkX 圖形物件
- 解析 JSON 格式的邊中繼資料
- 重建邊識別碼到中繼資料的對應關係

#### 邊中繼資料轉換 (Edge Metadata Conversion)
將 JSON 中的字串形式邊識別碼轉換回元組格式：
- 解析 `src→dst#key` 格式的邊識別碼
- 重建 `(src, dst, key)` 元組作為字典鍵值
- 確保資料結構與圖形物件的一致性

#### 預測資料整合 (Prediction Data Integration)
支援多種預測檔案格式的自動識別和載入：
- **REAPr格式**: `LineID`, `Type`, `Score` 欄位
- **通用格式**: `line_id`, `prediction`/`is_malicious`/`label` 欄位
- 自動篩選高分或標記為惡意的條目

### 3. 智慧序列偵測 (Intelligent Sequence Detection)
調用 `find_sequence_groups()` 執行進階的攻擊模式分析：

#### 偵測參數 (Detection Parameters)
- **最大間隔 (max_gap)**: 1 (序列中允許的操作間隔)
- **模式庫**: 使用預定義的 `ATTACK_SEQUENCE_PATTERNS`
- **目標檔案**: 用於特定的偵錯和最佳化

#### 偵測結果展示 (Detection Results Display)
提供前5個偵測到的群組的詳細資訊：
- 群組識別碼和模式名稱
- 置信度分數和完整性評估
- 實際匹配的操作與預期模式的對比
- 視覺化顏色分配

### 4. 進階視覺化生成 (Advanced Visualization Generation)
調用 `create_sequence_grouped_visualization()` 創建專門的序列分析視覺化：

#### 視覺化特色 (Visualization Features)
- **模式顏色編碼**: 每種攻擊模式使用專屬顏色
- **序列群組標記**: 相同序列的邊使用統一顏色
- **惡意行為高亮**: 整合預測資料的惡意標記
- **未分類邊處理**: 灰色顯示不屬於任何模式的邊

### 5. 統計分析與報告 (Statistical Analysis and Reporting)

#### 覆蓋率統計 (Coverage Statistics)
- **群組化覆蓋率**: 計算被歸類到攻擊模式的邊的百分比
- **未群組化邊數**: 統計未匹配任何模式的邊
- **整體分析效果**: 評估模式偵測的成效

#### 模式分佈報告 (Pattern Distribution Report)
- **按模式統計**: 每種攻擊模式匹配的邊數量
- **重要性排序**: 按邊數量排序的模式重要性
- **百分比分析**: 各模式在整體活動中的占比

### 6. 異常處理與驗證 (Exception Handling and Validation)
- 完整的錯誤捕捉和報告機制
- 檔案存在性的多重驗證
- 詳細的堆疊追蹤資訊用於偵錯

## 分析價值 (Analytical Value)

### 安全分析應用 (Security Analysis Applications)
- **威脅狩獵 (Threat Hunting)**: 識別複雜的攻擊鏈
- **事件回應 (Incident Response)**: 快速理解攻擊序列
- **模式學習 (Pattern Learning)**: 發現新的攻擊行為模式
- **風險評估 (Risk Assessment)**: 量化攻擊活動的嚴重程度

### 視覺化優勢 (Visualization Advantages)
- **直觀理解**: 複雜的日誌資料轉化為視覺化圖形
- **互動探索**: 支援動態篩選和詳細查看
- **模式識別**: 自動標記和分類攻擊行為
- **關聯分析**: 展示事件之間的時間和邏輯關係

這個深度分析功能為網路安全專家提供了強大的工具，能夠從大量的系統日誌中快速識別和分析潛在的安全威脅。

In [7]:

# Sequence Grouping

target_file = "cfb61005899996469ae3023796792ca5"  # Change this to your target file
graph_file = os.path.join(GRAPH_PATH, target_file, f"{target_file}.pkl")


graph_basename = os.path.basename(os.path.dirname(graph_file))
graph_dir = os.path.dirname(graph_file)
edge_metadata_file = os.path.join(graph_dir, f"{graph_basename}_edge_metadata.json")
predictions_file = f"Caldera_Ability_Predictions/{graph_basename}.csv"

if os.path.exists(graph_file) and os.path.exists(edge_metadata_file):
    
    # Load the graph and metadata
    G = pickle.load(open(graph_file, "rb"))
    with open(edge_metadata_file, "r") as fp:
        edge_metadata_json = json.load(fp)
    
    # Convert JSON edge metadata back to the expected format
    edge_metadata = {}
    for edge_id, data in edge_metadata_json.items():
        parts = edge_id.split('→')
        if len(parts) == 2:
            src = parts[0]
            dst_key = parts[1].split('#')
            if len(dst_key) == 2:
                dst = dst_key[0]
                key = int(dst_key[1]) if dst_key[1].isdigit() else dst_key[1]
                edge_metadata[(src, dst, key)] = data
    
    # Load predictions if available
    MALICIOUS_SPECS = []
    if os.path.exists(predictions_file):
        predictions_df = pd.read_csv(predictions_file)
        
        if 'LineID' in predictions_df.columns and 'Type' in predictions_df.columns and 'Score' in predictions_df.columns:
            malicious_rows = predictions_df[predictions_df['Score'] >= 1.0]
            for _, row in malicious_rows.iterrows():
                line_id = str(row['LineID'])
                prediction_type = row['Type'].lower()
                if prediction_type in ['src', 'dst']:
                    MALICIOUS_SPECS.append((line_id, prediction_type))
                else:
                    MALICIOUS_SPECS.append((line_id, "both"))

    
    sequence_groups = find_sequence_groups(
        G, edge_metadata, ATTACK_SEQUENCE_PATTERNS, 
        max_gap=1, target_file=target_file
    )
    
    # Show details of detected groups
    if sequence_groups:
        print(f"\n📋 DETECTED SEQUENCE GROUPS:")
        for group_id, group_info in list(sequence_groups.items())[:5]:  # Show first 5 groups
            print(f"\n   Group {group_id}: {group_info['pattern'].name}")
            print(f"      Description: {group_info['pattern'].description}")
            print(f"      Confidence: {group_info['confidence']:.2f}")
            print(f"      Edges: {len(group_info['edges'])}")
            print(f"      Operations: {', '.join(group_info['matched_operations'][:3])}{'...' if len(group_info['matched_operations']) > 3 else ''}")
            print(f"      Expected pattern: {', '.join(group_info['pattern'].operations)}")
            print(f"      Strict order: {group_info['pattern'].strict_order}")
            print(f"      Completeness: {group_info.get('completeness', 0):.2f}")
            print(f"      Color: {group_info['pattern'].color}")

    try:
        output_path = os.path.join(graph_dir, f"sequence_grouped_{graph_basename}.html")
        viz_path, groups = create_sequence_grouped_visualization(
            G, edge_metadata, MALICIOUS_SPECS, ATTACK_SEQUENCE_PATTERNS, output_path, sequence_groups
        )
        
        
        # Summary of what was found
        if groups:
            total_grouped_edges = sum(len(g['edges']) for g in groups.values())
            total_edges = G.number_of_edges()
            coverage = (total_grouped_edges / total_edges) * 100
            
            print(f"\n📊 Sequence Coverage:")
            print(f"   Grouped edges: {total_grouped_edges}/{total_edges} ({coverage:.1f}%)")
            print(f"   Ungrouped edges: {total_edges - total_grouped_edges} (shown in gray)")
            
            # Show pattern distribution
            pattern_counts = defaultdict(int)
            for group_info in groups.values():
                pattern_counts[group_info['pattern'].name] += len(group_info['edges'])
            
            print(f"\n🎯 Pattern Distribution:")
            for pattern_name, edge_count in sorted(pattern_counts.items(), key=lambda x: x[1], reverse=True):
                percentage = (edge_count / total_edges) * 100
                print(f"   {pattern_name}: {edge_count} edges ({percentage:.1f}%)")
        
    except Exception as e:
        print(f"❌ Error creating sequence visualization: {e}")
        import traceback
        traceback.print_exc()

else:
    print(f"❌ Required files not found for {target_file}")
    print(f"   Graph file: {graph_file}")
    print(f"   Metadata file: {edge_metadata_file}")


❌ Required files not found for cfb61005899996469ae3023796792ca5
   Graph file: Graphs\cfb61005899996469ae3023796792ca5\cfb61005899996469ae3023796792ca5.pkl
   Metadata file: Graphs\cfb61005899996469ae3023796792ca5\cfb61005899996469ae3023796792ca5_edge_metadata.json
